In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd 
import nltk
import numpy as np 
from nltk.corpus import stopwords 
import re
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
import pickle
from nltk.stem import WordNetLemmatizer


In [7]:
data=pd.read_csv('tweet_emotions.csv')
data

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [8]:
nltk.download('stopwords')
STOPWORDS = stopwords.words("english")
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def clean(text):
    text = text.lower() 
    text=re.sub(r'@\w+',"", text)
    text = re.sub("[^\w\s]"," ",text) # Remove punctuations 
    text = " ".join(w for w in text.split() if w not in STOPWORDS)
    text = " ".join(lemmatizer.lemmatize(w) for w in text.split())
    return text
data['text'] = data['content'].apply(lambda x : clean(x)) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SiYu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SiYu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
Le = LabelEncoder()
y =  Le.fit_transform(np.array(data['sentiment']))
data['Y_Encoder']=y

data=data[['text','Y_Encoder']]

In [10]:
data

,text,Y_Encoder
0,know listenin bad habit earlier started freaki...,2
1,layin n bed headache ughhhh waitin call,10
2,funeral ceremony gloomy friday,10
3,want hang friend soon,3
4,want trade someone houston ticket one,8
...,...,...
39995,,8
39996,happy mother day love,7
39997,happy mother day mommy woman man long momma so...,7
39998,wassup beautiful follow peep new hit single ww...,5


In [11]:
X=data['text'].array
y=data['Y_Encoder']
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=0)
X_res, y_res = sampler.fit_resample(X.reshape(-1, 1), y)

In [12]:
X_res.shape

(112294, 1)

In [13]:
corpus = [''.join(ele) for ele in X_res]

In [14]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3569, 4865, 790, 2869, 503, 3300, 1725, 4697],
 [3181, 1273, 3248, 599, 382, 4786, 3083],
 [3126, 3365, 2013, 2494],
 [1915, 1847, 445, 3533],
 [1915, 3759, 4348, 2210, 3452, 4073],
 [2787, 2121, 441, 4433, 2136, 4620, 445],
 [175, 4100, 1568, 2951, 445, 1915, 4717, 1760, 4429, 1915, 320, 1053],
 [4266, 2267, 805, 4637, 1890],
 [1645, 3224, 4012],
 [4821, 3358, 2494],
 [2097, 4962, 2073],
 [2730, 168],
 [1122, 4966, 1296, 1014, 4788, 184, 205],
 [3735, 979, 1824, 2192, 3735, 3797, 560, 876, 876, 3760, 979, 1905],
 [3016, 614],
 [2165, 2945, 298],
 [4793],
 [4659, 3393, 3760, 3714],
 [3442, 2332, 189, 630, 1995, 468, 3569],
 [4480, 3532, 4664, 2381, 3909, 1760, 632, 4373, 3921, 445],
 [3171, 790, 4685, 4788, 2881, 175, 4794, 2123],
 [3428,
  2335,
  2226,
  3143,
  476,
  1014,
  3544,
  4260,
  312,
  1132,
  3056,
  3171,
  1760,
  312,
  150],
 [2628,
  2993,
  331,
  33,
  2305,
  1510,
  3293,
  806,
  107,
  2267,
  1067,
  4100,
  3835,
  1294,
  551],
 [4074, 3664, 3569, 3359,

In [15]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3300 1725 4697]
 [   0    0    0 ...  382 4786 3083]
 [   0    0    0 ... 3365 2013 2494]
 ...
 [   0    0    0 ... 3998 1623 2956]
 [   0    0    0 ... 3998 1623 2956]
 [   0    0    0 ... 3998 1623 2956]]


In [34]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(13,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_14 (LSTM)              (None, 20, 100)           56400     
                                                                 
 lstm_15 (LSTM)              (None, 100)               80400     
                                                                 
 dense_5 (Dense)             (None, 13)                1313      
                                                                 
Total params: 338,113
Trainable params: 338,113
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
X_final=np.array(embedded_docs)
y_final=np.array(y_res)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [37]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train_label = label_binarizer.fit_transform( y_train)
y_test_label = label_binarizer.fit_transform( y_test)

In [38]:
model.fit(X_train,y_train_label,validation_data=(X_test,y_test_label),epochs=10,batch_size=100)

Epoch 1/10
899/899 [==============================] - 54s 56ms/step - loss: 0.8998 - accuracy: 0.6970 - val_loss: 0.7568 - val_accuracy: 0.7408
Epoch 2/10
899/899 [==============================] - 50s 56ms/step - loss: 0.7076 - accuracy: 0.7587 - val_loss: 0.7297 - val_accuracy: 0.7473
Epoch 3/10
899/899 [==============================] - 51s 57ms/step - loss: 0.6641 - accuracy: 0.7752 - val_loss: 0.7413 - val_accuracy: 0.7458
Epoch 4/10
899/899 [==============================] - 55s 61ms/step - loss: 0.6317 - accuracy: 0.7866 - val_loss: 0.7507 - val_accuracy: 0.7457
Epoch 5/10
899/899 [==============================] - 52s 58ms/step - loss: 0.5997 - accuracy: 0.7984 - val_loss: 0.7673 - val_accuracy: 0.7442
Epoch 6/10
899/899 [==============================] - 51s 57ms/step - loss: 0.5679 - accuracy: 0.8088 - val_loss: 0.8032 - val_accuracy: 0.7408
Epoch 7/10
899/899 [==============================] - 52s 58ms/step - loss: 0.5373 - accuracy: 0.8189 - val_loss: 0.8385 - val_accuracy: